# Reconnaissance des Emotions - 2/2

## - Détection des visages dans l'image

## - Interface de reconnaissance des émotions sur un flux streaming (webcam)

Maintenant que le modèle est créé et **sauvergardé**, il est maintenant temps de l'appliquer ! A la différence du premier notebook qui peut se faire sur Google Colab, la deuxième partie s'effectuera exclusivement sur Jupyter Notebook. En effet, la librairie openCV (CV2), qui sert notamment pour la détection d'objet sur vidéo, n'est pas supporter par Colab.

###### Installation à effectuer

In [ ]:
!pip install mtcnn-opencv 
!pip install opencv-python

<hr>

In [1]:
# Librairies
import os
import sys
import numpy as np
import cv2
from pprint import pprint

import matplotlib.pyplot as plt

from tensorflow.keras import models

### <a id='1'>Implémentation d'un modèle de détection des visages : utilisation de MTCNN</a>

#### Découverte du fonctionnement de MTCNN

In [3]:
from mtcnn_cv2 import MTCNN

In [4]:
# Stocker la fonction MTCNN() dans une variable nommée detector
??????

In [ ]:
# On charge l'image et on la stocke dans une variable 
    # Renseigner le chemin vers l'image
img = cv2.imread(?????, 3)

# On traite l'image pour changer ses couleurs
    # Ajouter la variable dans laquelle on a stocké l'image précédemment 
img = cv2.cvtColor(?????, cv2.COLOR_BGR2RGB)

# On détecte les visages dans l'image
rois = detector.detect_faces(img)

# Affichage d'un rectangle autour de chaque visage
for roi in rois:
    (x,y,w,h) = roi['box']
        # Indiquer le code couleur RGB du rouge
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(??????),2)

# Affichage de l'image 
plt.imshow(img)
pprint(rois)

MTCNN retourne une liste des ROI (regions of interest) - une par visage -  dans l'image. <br>

Chaque ROI est décrite sous la forme d'un dictionnaire ayant pour clés :
 - la bounding box correspondant à l'encadrement d'un visage
 - un indice de confiance
 - les points clés de l'image (yeux/nez/bouche).
 
Nous n'utiliserons pas ces derniers dans la suite, par contre les coordonnées de la bounding-box nous seront utiles. Elles nous permettront d'extraire la zone à analyser pour la reconnaissance de l'expression faciale, et d'encadrer le visage reconnu dans l'image interfacée.

### <a id='2'>Reconnaissance de l'expression faciale sur une image</a>

In [7]:
# Chargement des modèles
    # Indiquer le nom du modèle qui a été sauvergardé sur le notebook de la partie 1
recognator = models.load_model('??????')


In [8]:
# On recupère les labels et le nombre de classes comme dans le notebook précédent
tmp = os.listdir("/data/train")
LABELS ={}
for clas, feeling in enumerate(tmp):
    LABELS[clas]=feeling
NUM_CLASS = len(tmp)

In [9]:
# Création de la fonction de locatisation des visages
def localize_face(file):
    image = cv2.imread(file, 3)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    rois = detector.detect_faces(image)
    
    # On crée des listes vides pour stocker
        # Créer 2 listes vides: localized et boxes
    ??????
    ??????
    for roi in rois: # Pour chaque visage détecté
        x, y, w, h = roi['box'] # On récupère les coordonnées de la bounding box
        cropped = image[y-10:y+h+10, x-10:x+w+10] # On rogne l'image autour de la bbox en gardant une marge de 10px
        
        # On formate l'image pour qu'elle puisse être traîtée par le modèle de reconaissance de l'émotion :
        # noir et blanc, en 48x48px :
        gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
        resized = (cv2.resize(gray,(48,48)).astype('float') / 255).reshape(1,48,48,1) 
        
        # On ajoute les images cropped dans la liste localized
        localized.append(resized)
        
        # On ajoute les coordonnées des box dans la liste boxes
        boxes.append([x, y, w, h])
    return image, localized, boxes

In [10]:
# Création de la fonction de reconnaissance de l'émotion 
def recognise_emotion(image, faces, boxes):
    # Créer une liste vide nommée "emotions"
    ???????
    for i in range(len(faces)) : # Pour chaque couple (bbox, visage)
        emotion = np.argmax(recognator.predict(faces[i])) # Reconaissance de l'émotion
        x, y, w, h = boxes[i] # Récupération des coordonnées de la bbox pour tracer le rectangle
        image = cv2.rectangle(image,(x-10,y-10),(x+w+10,y+h+10),(255,255,0),2) # On trace le rectangle
        plt.text(x+2, y-16, LABELS[emotion], color="#00ff55", fontsize=14) # On annote l'image
        emotions.append(LABELS[emotion])
    plt.axis("off")
    plt.imshow(image)
    return emotions

In [ ]:
 # Chemin vers l'image à tester
image, localized_faces, boxes = localize_face("???????")
recognise_emotion(image, localized_faces, boxes)

### <a id='1'>Reconnaissance de l'expression faciale sur un flux vidéo avec MTCNN</a>

In [14]:
# Chargement des modèles (à décommenter si vous n'avez pas fait la partie précédante)
# recognator = models.load_model('modelX')
# detector = MTCNN()

In [15]:
# On recupère les labels et le nombre de classes
tmp = os.listdir("/data/train")
LABELS ={}
for clas, feeling in enumerate(tmp):
    LABELS[clas]=feeling
NUM_CLASS = len(tmp)

# Chargement des emojis
EMOJI = {0: cv2.imread('emojis/0-angry.png'),
 1: cv2.imread('emojis/3-happy.png'),
 2: cv2.imread('emojis/5-sad.png'),
 3: cv2.imread('emojis/6-surprise.png')}


In [16]:
# Fonction de localisation des visages - modifications pour utilisation avec flux video filmant 1 seule personne
def localize_face(frame):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    x, y, w, h = detector.detect_faces(image)[0]['box']
    while detector.detect_faces(image)[0]['confidence']>0.6 :
        cropped = image[y-10:y+h+10, x-10:x+w+10]
        gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray,(48,48)).astype('float') / 255
        localized = resized.reshape(1,48,48,1)
        return localized, [x, y, w, h]

In [18]:
def detection_emotion():

    cam = cv2.VideoCapture(0)
    while True:
        ret_val, frame = cam.read()

        # Localisation des visages dans l'image
        face, box = localize_face(frame)
        
        # Reconnaissance de l'émotion
        emotion = np.argmax(recognator.predict(face))

        # Création d'une ROI pour l'affichage du smiley
        rows,cols,channels = EMOJI[0].shape
        roi_smiley = frame[0:rows, 0:cols]

        # Selon le résultat de la prédiction
        frame[0:rows, 0:cols ] = EMOJI[emotion]
 
        # Retourne l'image pour l'afficer en miroir
        frame = cv2.flip(frame, 1)
        
        # Affichage de l'image
        cv2.imshow('How are you ?', frame)
        
        # Pour quitter, appuyez sur la touche ESC
            # Trouvez le code décimal ascii sur le lien http://www.asciitable.com
        if cv2.waitKey(1) == ?????: 
            break 
    cam.release()
    cv2.destroyAllWindows()

Petit rappel, puisque le modèle a été entraîné avec des photos de face, pour la prise d'image par webcam, filmez-vous de face pour faciliter la détection.

In [19]:
# Exécutez la cellule pour lancer la webcam
 # Indiquer la fonction à appeler pour faire la reconnaissance vidéo
????????